### Define variables

In [1]:
arr_model = ['vgg16', 'resnet18']
arr_data  = ['cifar10', 'cifar100']
arr_lr    = ['0.20', '0.10', '0.05', '0.01']
arr_l     = ['16', '8', '4', '2']

In [2]:
path_train = '/cluster/home/pintoa/ai-research-snn/logs/20220523/train'
path_eval  = '/cluster/home/pintoa/ai-research-snn/logs/20220523/eval'

In [3]:
cmd1 = 'bsub -n 4 -W 08:00 -o {path}/{id}.out -R "rusage[mem=2048, ngpus_excl_p=1]" -R "select[gpu_mtotal0>=10240]"'
cmd2 = 'python main.py train --model={model} --data={data} --lr={lr} --l={l} --id={id}'
cmd3 = 'python main.py test --model={model} --data={data} --id={id} --mode={mode}'

### Produce `train.sh`

In [4]:
with open('train.sh', 'w') as f:
    f.write('#!/bin/bash\n')
    for model in arr_model:
        for data in arr_data:
            for lr in arr_lr:
                for l in arr_l:
                    id = '{model}_{data}_LR{lr}_L{l}'.format(model=model, data=data, lr=lr[2:], l=l)
                    c1 = cmd1.format(path=path_train, id=id)
                    cmd = c1 + ' ' + cmd2.format(model=model, data=data, lr=lr, l=l, id=id)
                    f.write(cmd + '\n')

### Produce `eval.sh`

In [4]:
mode = ['ann', 'snn']

In [5]:
ids = []
for model in arr_model:
    for data in arr_data:
        for lr in arr_lr:
            for l in arr_l:
                ids.append('{model}_{data}_LR{lr}_L{l}'.format(model=model, lr=lr[2:], data=data, l=l))

In [6]:
with open('eval.sh', 'w') as f:
    f.write('#!/bin/bash\n')
    for model in arr_model:
        for data in arr_data:
            for lr in arr_lr:
                for l in arr_l:
                    for m in mode:
                        saved_model = '{model}_LR{lr}_{data}_L{l}'.format(model=model, data=data, lr=lr[2:], l=l)
                        nl = '0' + str(l) if l != '16' else str(l)
                        new_id = '{model}_{data}_LR{lr}_L{l}_{mode}'.format(model=model, data=data, lr=lr[2:], l=nl, mode=m)
                        c1 = cmd1.format(path=path_eval, id=new_id)
                        cmd = c1 + ' ' + cmd3.format(model=model, data=data, id=saved_model, mode=m)
                        if m == 'snn':
                            cmd += ' --t=128'
                        f.write(cmd + '\n')